In [15]:

from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.models.bars import Bar

from flask import Flask, request, jsonify, session

from config import ALPACA_SECRET_KEY, ALPACA_PUBLIC_KEY, FMP_API_KEY
from urllib.request import urlopen
from datetime import datetime, timezone, timedelta

import talib as ta
import numpy as np
import pandas as pd
import logging
import json
import pickle

# ALL INDICATORS

# overlap_studies = [BBANDS_indicator, DEMA_indicator, EMA_indicator, HT_TRENDLINE_indicator, KAMA_indicator, MA_indicator, MAMA_indicator, MAVP_indicator, MIDPOINT_indicator, MIDPRICE_indicator, SAR_indicator, SAREXT_indicator, SMA_indicator, T3_indicator, TEMA_indicator, TRIMA_indicator, WMA_indicator]
# momentum_indicators = [ADX_indicator, ADXR_indicator, APO_indicator, AROON_indicator, AROONOSC_indicator, BOP_indicator, CCI_indicator, CMO_indicator, DX_indicator, MACD_indicator, MACDEXT_indicator, MACDFIX_indicator, MFI_indicator, MINUS_DI_indicator, MINUS_DM_indicator, MOM_indicator, PLUS_DI_indicator, PLUS_DM_indicator, PPO_indicator, ROC_indicator, ROCP_indicator, ROCR_indicator, ROCR100_indicator, RSI_indicator, STOCH_indicator, STOCHF_indicator, STOCHRSI_indicator, TRIX_indicator, ULTOSC_indicator, WILLR_indicator]
# volume_indicators = [AD_indicator, ADOSC_indicator, OBV_indicator]
# cycle_indicators = [HT_DCPERIOD_indicator, HT_DCPHASE_indicator, HT_PHASOR_indicator, HT_SINE_indicator, HT_TRENDMODE_indicator]
# price_transforms = [AVGPRICE_indicator, MEDPRICE_indicator, TYPPRICE_indicator, WCLPRICE_indicator]
# volatility_indicators = [ATR_indicator, NATR_indicator, TRANGE_indicator]
# pattern_recognition = [CDL2CROWS_indicator, CDL3BLACKCROWS_indicator, CDL3INSIDE_indicator, CDL3LINESTRIKE_indicator, CDL3OUTSIDE_indicator, CDL3STARSINSOUTH_indicator, CDL3WHITESOLDIERS_indicator, CDLABANDONEDBABY_indicator, CDLADVANCEBLOCK_indicator, CDLBELTHOLD_indicator, CDLBREAKAWAY_indicator, CDLCLOSINGMARUBOZU_indicator, CDLCONCEALBABYSWALL_indicator, CDLCOUNTERATTACK_indicator, CDLDARKCLOUDCOVER_indicator, CDLDOJI_indicator, CDLDOJISTAR_indicator, CDLDRAGONFLYDOJI_indicator, CDLENGULFING_indicator, CDLEVENINGDOJISTAR_indicator, CDLEVENINGSTAR_indicator, CDLGAPSIDESIDEWHITE_indicator, CDLGRAVESTONEDOJI_indicator, CDLHAMMER_indicator, CDLHANGINGMAN_indicator, CDLHARAMI_indicator, CDLHARAMICROSS_indicator, CDLHIGHWAVE_indicator, CDLHIKKAKE_indicator, CDLHIKKAKEMOD_indicator, CDLHOMINGPIGEON_indicator, CDLIDENTICAL3CROWS_indicator, CDLINNECK_indicator, CDLINVERTEDHAMMER_indicator, CDLKICKING_indicator, CDLKICKINGBYLENGTH_indicator, CDLLADDERBOTTOM_indicator, CDLLONGLEGGEDDOJI_indicator, CDLLONGLINE_indicator, CDLMARUBOZU_indicator, CDLMATCHINGLOW_indicator, CDLMATHOLD_indicator, CDLMORNINGDOJISTAR_indicator, CDLMORNINGSTAR_indicator, CDLONNECK_indicator, CDLPIERCING_indicator, CDLRICKSHAWMAN_indicator, CDLRISEFALL3METHODS_indicator, CDLSEPARATINGLINES_indicator, CDLSHOOTINGSTAR_indicator, CDLSHORTLINE_indicator, CDLSPINNINGTOP_indicator, CDLSTALLEDPATTERN_indicator, CDLSTICKSANDWICH_indicator, CDLTAKURI_indicator, CDLTASUKIGAP_indicator, CDLTHRUSTING_indicator, CDLTRISTAR_indicator, CDLUNIQUE3RIVER_indicator, CDLUPSIDEGAP2CROWS_indicator, CDLXSIDEGAP3METHODS_indicator]
# statistical_functions = [BETA_indicator, CORREL_indicator, LINEARREG_indicator, LINEARREG_ANGLE_indicator, LINEARREG_INTERCEPT_indicator, LINEARREG_SLOPE_indicator, STDDEV_indicator, TSF_indicator, VAR_indicator]




```
$ tar -xzf ta-lib-0.6.4-src.tar.gz
$ cd ta-lib-0.6.4/
$ ./configure --prefix=/usr
$ make
$ sudo make install
```

next move the folder into your site packages folder, then do

```
python3 -m pip install TA-lib
```

In [3]:
url = f"https://financialmodelingprep.com/api/v3/nasdaq_constituent?apikey={FMP_API_KEY}"
response = urlopen(url)
data = response.read().decode("utf-8")
data = json.loads(data)

data

[{'symbol': 'ADBE',
  'name': 'Adobe Inc.',
  'sector': 'Technology',
  'subSector': 'Software - Infrastructure',
  'headQuarter': 'San Jose, CA',
  'dateFirstAdded': None,
  'cik': '0000796343',
  'founded': '1982-12-01'},
 {'symbol': 'AMAT',
  'name': 'Applied Materials',
  'sector': 'Technology',
  'subSector': 'Semiconductors',
  'headQuarter': 'Santa Clara, CA',
  'dateFirstAdded': None,
  'cik': '0000006951',
  'founded': '1967-11-10'},
 {'symbol': 'CSCO',
  'name': 'Cisco',
  'sector': 'Technology',
  'subSector': 'Communication Equipment',
  'headQuarter': 'San Jose, CA',
  'dateFirstAdded': None,
  'cik': '0000858877',
  'founded': '1984-12-10'},
 {'symbol': 'FAST',
  'name': 'Fastenal',
  'sector': 'Industrials',
  'subSector': 'Industrial - Distribution',
  'headQuarter': 'Winona, MN',
  'dateFirstAdded': None,
  'cik': '0000815556',
  'founded': '1967-01-01'},
 {'symbol': 'MSFT',
  'name': 'Microsoft',
  'sector': 'Technology',
  'subSector': 'Software - Infrastructure',
  

In [4]:
# Save the data as a pickle file
with open("nasdaq_data.pkl", "wb") as file:
    pickle.dump(data, file)

In [5]:
# Save the data as a pickle file
with open("nasdaq_data.pkl", "rb") as file:
    data = pickle.load(file)

In [6]:
NDQ100 = []
for element in data:
    NDQ100.append(element["symbol"])

In [7]:
client = StockHistoricalDataClient(ALPACA_PUBLIC_KEY, ALPACA_SECRET_KEY)

# client.get_stock_latest_quote(params)
start_day = datetime.now(tz=timezone.utc) - timedelta(days=1)
end_day = datetime.now(tz=timezone.utc)

params = StockBarsRequest(symbol_or_symbols=NDQ100, start=start_day, timeframe=TimeFrame.Minute, limit=10000000)

data = client.get_stock_bars(params)

In [8]:
data_iter = iter(data)
(_, unwrapped_data) = next(data_iter)

In [9]:
dfs = {}
for ticker in NDQ100:
    # extracts a ticker from the list of 100
    bars = unwrapped_data[ticker]
    # Convert to DataFrame
    df = pd.DataFrame([bar.__dict__ for bar in bars])
    dfs[ticker] = df

In [10]:
# indicator example using TAlib
prices = dfs["AAPL"]["close"].to_numpy()
aapl_sma = ta.SMA(prices, 60)

In [28]:
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NVDA']
dfs = {}
for ticker in tickers:
    bars = unwrapped_data[ticker]
    dfs[ticker] = bars

json.dumps(dfs, default=str)

'{"AAPL": ["symbol=\'AAPL\' timestamp=datetime.datetime(2025, 3, 12, 14, 11, tzinfo=TzInfo(UTC)) open=221.45 high=221.6 low=221.1 close=221.1178 volume=192082.0 trade_count=2332.0 vwap=221.356578", "symbol=\'AAPL\' timestamp=datetime.datetime(2025, 3, 12, 14, 12, tzinfo=TzInfo(UTC)) open=221.11 high=221.11 low=220.6 close=220.79 volume=153302.0 trade_count=2508.0 vwap=220.774568", "symbol=\'AAPL\' timestamp=datetime.datetime(2025, 3, 12, 14, 13, tzinfo=TzInfo(UTC)) open=220.8 high=220.87 low=220.6216 close=220.72 volume=81511.0 trade_count=1431.0 vwap=220.745781", "symbol=\'AAPL\' timestamp=datetime.datetime(2025, 3, 12, 14, 14, tzinfo=TzInfo(UTC)) open=220.745 high=220.91 low=220.65 close=220.73 volume=138541.0 trade_count=1499.0 vwap=220.739213", "symbol=\'AAPL\' timestamp=datetime.datetime(2025, 3, 12, 14, 15, tzinfo=TzInfo(UTC)) open=220.73 high=220.798 low=220.37 close=220.5 volume=159934.0 trade_count=1957.0 vwap=220.572716", "symbol=\'AAPL\' timestamp=datetime.datetime(2025, 3, 

alpaca.data.models.bars.Bar

In [ ]:
datetime.now(tz=timezone.utc)

datetime.datetime(2025, 3, 13, 13, 38, 27, 747500, tzinfo=datetime.timezone.utc)

In [ ]:
def skip_weekends(curr_date):
    curr_date = datetime.now(tz=timezone.utc)
    # sunday
    if curr_date.weekday() == 0:
        return curr_date - timedelta(days = 2)
    # saturday
    elif curr_date.weekday() == 6:
        return curr_date - timedelta(days = 1)
    
    return curr_date

